# SVM classification of surface residues

This notebook documents my work on SVM classification of surface residues of receptors in peptide-protein interactions.
The data is read directly from Dana's early work on PeptiDB, analyzing surface residues in that data set using various tools.

First, import the necessary Python modules for analysis, primarily SciKit-Learn (sklearn) used for machine learning and statistical analysis.

In [ ]:
%pylab inline

In [ ]:
import re
import os
import numpy as np
from scipy import interp
import pylab as pl
import hashlib

# caching/serialization libraries
import pickle
import joblib

from datetime import datetime
from texttable import Texttable
from IPython.core.display import Latex, HTML, display

from itertools import combinations, chain
from treedict import TreeDict

from sklearn import svm, datasets, metrics, cross_validation, preprocessing
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import StratifiedKFold, KFold, LeaveOneLabelOut

import pandas as pd

import data
import config

In [ ]:
THESIS_SRC = '$HOME/lab/msc-thesis/source'

In [ ]:
pd.options.display.width = 250

## Preliminary SVM for feature weights

### AUC and feature weight over all features, using cross-validation data

Start by defining a K-fold partition of the data, plus a SV classifier. 
The partition can be either a random partition, a random stratified partition, or by label.

In our case, we use 4-fold leave-one-label-out cross-validation. 
This means we divide the data set of residues into 4 disjoint subsets, such that all the residues of any one receptor are in the same subset. 
The subsets are roughly similar in size, but cannot be guaranteed to be an equal partition of the data.

The partition is accomplished by assigning an integer label to each residue that is the ASCII value of that residue's PDB ID, modulo the number of subsets we want (in our case, 4). 
The ASCII value of a string is the sum of ASCII numbers of each of its characters. 
Therefore, all residues of the same receptor will have the same PDB ID, hence the same ASCII value, hence the same label.

In [ ]:
%%javascript
Jupyter.utils.load_extensions('gist')

In [ ]:
reload(data)

In [ ]:
original_features = [
                     'FTMap.cs_size',
                     'FTMap.num_nearby_cs',
                     'ConSurf.Conservation-score',
                     'FPocket.Real-volume-(approximation)',
                     'Physicochemical.Polar',
                     'Physicochemical.Hydrogen-bonding',
                    ]
original_feature_set = data.FeatureSet(original_features, original_features)

In [ ]:
data_tdict = TreeDict()
for context in ('bound', 'unbound'):
    for version in ('new', 'new_reduced', 'old',):
        data_tdict.update({'.'.join([version, context]) : 
        data.prepDataSet('%s.data.%s.csv' % (context, version), 
                         dataset_name='peptalk.{}.{}'.format(context, version),
                        )
                 })
datasets = data_tdict.interactiveTree()

In [ ]:
d = datasets.old.bound

In [ ]:
n_samples, n_features = d.X.shape

# Initialize classifier with crossvalidation
k_folds = 4

#cv = KFold(len(y), k=k_folds)
#cv = StratifiedKFold(y, k=k_folds)

hash_to_k = lambda s: (long(hashlib.sha1(s).hexdigest(), 16) % k_folds)
pdb_labels = np.array([hash_to_k(s) for s in d.pdbs])
cv = LeaveOneLabelOut(labels=pdb_labels)

#classifier = svm.SVC(kernel='linear', probability=True, class_weight='auto')
classifier = svm.LinearSVC(class_weight='auto', 
                           dual=True, 
                           loss='l1', 
                           )

Here we calculate performance statistics of the classifier, such that for each of the labels we defined above, we:

1. Train (or fit) the classifier on other labels, and record the feature weights for the trained model.
2. Predict the likelihood of residues from the training set (i.e. other labels) to be binders, and from the test set (the current label).
3. Using these predictions, calculate ROC curves for the training and test sets, and record the AUC for each of them.

Finally, print all statistics detailed above in a table, where each row represents one label. 
Additionally, print mean values for all columns (feature weights, AUCs).

In [ ]:
feat_weights = np.zeros((len(cv), n_features))
test_aucs = np.zeros(len(cv))
train_aucs = np.zeros(len(cv))

#clf_table = Texttable(max_width=160)
#clf_table.set_deco(Texttable.HEADER | Texttable.VLINES)
#clf_table.set_cols_dtype(list('t' + 'a'*n_features + 'cc'))
#clf_table.set_cols_align(list('l' + 'c'*n_features + 'cc'))
#clf_table.set_precision(4)
#clf_table.header(['CV subset'] + feature_names + ['AUC (training)', 'AUC (testing)'])

mean_tpr = 0.0
mean_tpr_train = 0.0
fpr_grid = np.linspace(0, 1, 100)
all_tpr = []

print "Calculating CV: ",
for i, (train, test) in enumerate(cv):
    print i,
    classifier.fit(d.X[train,:], d.y[train])
    feat_weights[i] = classifier.coef_
    
    # Test on the training set
    tr_conf_scores = classifier.decision_function(d.X[train,:])
    tr_fpr, tr_tpr, tr_thresholds = roc_curve(d.y[train], tr_conf_scores)
    train_aucs[i] = auc(tr_fpr, tr_tpr)
    mean_tpr_train += interp(fpr_grid, tr_fpr, tr_tpr, left=0, right=1)
    
    #Test on the test set
    conf_scores = classifier.decision_function(d.X[test,:])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(d.y[test], conf_scores)
    test_aucs[i] = auc(fpr, tpr)
    mean_tpr += interp(fpr_grid, fpr, tpr, left=0, right=1)

    #clf_table.add_row(["CV %d" % i] + feat_weights[i].tolist() + [train_aucs[i], test_aucs[i]])

print 'Done.'
stats = np.c_[feat_weights, train_aucs, test_aucs]
cv_names = [['CV%d'%i] for i in range(len(cv))]
#for i in range(len(cv)):
#    clf_table.add_row(cv_names[i] + stats[i].tolist())
#clf_table.add_row(['Mean'] + stats.mean(axis=0).tolist())

mean_tpr /= len(cv)
mean_tpr[-1] = 1.0
mean_auc = auc(fpr_grid, mean_tpr)

mean_tpr_train /= len(cv)
mean_tpr_train[-1] = 1.0
mean_auc_train = auc(fpr_grid, mean_tpr_train)
#print 
#print clf_table.draw()

In [ ]:
pd.options.display.precision = 3

In [ ]:
t = pd.DataFrame(stats, columns=d.feature_set.features + ['AUC (training)', 'AUC (testing)'], index=['CV%d' % i for i in range(k_folds)])
#display(t.append(t.describe()))
#display(
#        Latex('Feature weights alone:'), 
#        t.ix[:,:-2].abs().T,
#        )

tbl_svm_coefs = t.append(t.describe().ix[1:3,:])

display(Latex('Entire stats table:'), 
        tbl_svm_coefs,#.T.abs().sort(columns=['mean',], ascending=False),
        )

#tbl_svm_coefs.to_csv(os.path.join(THESIS_SRC, '_tables', 'table-svm-coefs.csv'), float_format='%4.2f')
#display(t.describe())

## ROC curves of different classification configs

### Define a way to test these configurations:

Set some parameters for `matplotlib` (plotting library):

In [ ]:
rcParams = matplotlib.rc_params_from_file("/Users/assafl/dev/github/matplotlibrc-huyng/matplotlibrc")

In [ ]:
rcParams['text.usetex'] = False
rcParams['font.size'] = 10.0

In [ ]:
rcParams['figure.figsize'] = (12.0, 12.0)
rcParams['legend.fontsize'] = 'medium'
rcParams['legend.loc'] = 'best'
rcParams['lines.linewidth'] = 2

Test multiple configs in sequence and plot all ROC curves together:

In [ ]:
def testConfigs(configs, saveto=None):
    
    n_configs = len(configs)
    coefs_dict = {}
    
    #pl.subplot(111)    
    for i, c in enumerate(configs):
        clf = config.trainClassifier(c)
        coefs_dict[c.title] = dict(zip(c.feature_set.features, clf.coef_.tolist()[0]))
        conf_scores = clf.decision_function(c.testing.X)
        
        display(Latex("Calculating predictions on feature set: %s" % c.title))
        
        fpr, tpr, thresholds = roc_curve(c.testing.y, conf_scores)
        fpr_grid = np.linspace(0, 1, 100)
        tpr_interp = interp(fpr_grid, fpr, tpr, left=0, right=1)
        roc_auc = auc(fpr_grid, tpr_interp)
        
        is_delta = c.title.find('Delta')!=-1
        
        pl.plot(
                fpr_grid, tpr_interp, 
                label='%s (AUC = %0.2f)' % (c.title.replace('_',' '), roc_auc),
                linestyle='dashed' if is_delta else 'solid',
                )
    pl.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Random')
    
    pl.xlim([-0.05, 1.05])
    pl.ylim([-0.05, 1.05])
    pl.xlabel('False Positive Rate')
    pl.ylabel('True Positive Rate')
    #pl.title('Comparison of classifier configurations' + '\n'+
    #        'Mean ROC curves, performance measured on test set')
    pl.legend()#bbox_to_anchor=(1, 1), loc=2)
    if saveto:
        pl.savefig(saveto)
    pl.show()
    
    coefs_df = pd.DataFrame.from_dict(coefs_dict).T
    coefs_df.index.names = ['Configurations (by feature set)']
    #coefs_df.columns.names = ['Feature weights']
    display(coefs_df)

In [ ]:
reload(config)

### Analysis of features against the original six features in our SVM

In [ ]:
def compareSingleFeatures(training_set, testing_set, features, delta=False):
    tr_csv = training_set.csv_filename
    te_csv = testing_set.csv_filename
    
    full_fs = data.FeatureSet(features, features)
    configs = [config.createConfig(full_fs, train=tr_csv, test=te_csv)]
    
    for feature_singleton in combinations(features, 1):
        single_fs = data.FeatureSet(feature_singleton, features)
        partial_fs = single_fs.complement() if delta else single_fs
        
        configs.append(config.createConfig(partial_fs, train=tr_csv, test=te_csv))
    
    figure_filename = 'roc-configs_{version}_{scope}-{delta}.svg'.format(
                             scope='feature',
                             version=os.path.basename(tr_csv).split('.')[2],
                             delta='delta' if delta else 'single',
                             )
        
    clfs = config.trainConfigClassifiers(configs)
    testConfigs(configs, saveto=figure_filename)

Select the old version of the PeptiDB data, as it was generated by Dana:

In [ ]:
d = datasets.old

Now generate and compare two sets of configs:

1. Train each config on *one feature*.
2. Train each config on *all features but one* (aka $\Delta$ configs).

In [ ]:
rcParams['lines.linewidth'] = 3

In [ ]:
compareSingleFeatures(d.unbound, d.bound, d.bound.feature_set.features, delta=False)

In [ ]:
compareSingleFeatures(d.unbound, d.bound, d.bound.feature_set.features, delta=True)

### Analysis of protocols against all 48 features calculated recently

In [ ]:
def compareProtocolFeatures(training_set, testing_set, features, delta=False):
    tr_csv = training_set.csv_filename
    te_csv = testing_set.csv_filename
    
    protocols = set(feat.partition('.')[0] for feat in features)
    protocol_features = lambda protocol: filter(lambda feat: feat.startswith(protocol), features)
    
    full_fs = data.FeatureSet(features, features)
    configs = [config.createConfig(full_fs, train=tr_csv, test=te_csv)]
    
    for protocol in sorted(protocols):
        single_fs = data.FeatureSet(protocol_features(protocol), features, meta=protocol+' features')
        partial_fs = single_fs.complement() if delta else single_fs
        
        configs.append(config.createConfig(partial_fs, train=tr_csv, test=te_csv, ddg_cutoff=0))
        
    figure_filename = 'roc-configs_{version}_{scope}-{delta}.svg'.format(
                             scope='protocol',
                             version=os.path.basename(tr_csv).split('.')[2],
                             delta='delta' if delta else 'single',
                             )
    
    clfs = config.trainConfigClassifiers(configs)
    testConfigs(configs, saveto=figure_filename)

In [ ]:
d = datasets.old

In [ ]:
compareProtocolFeatures(d.unbound, d.bound, d.bound.feature_set.features, delta=False)

In [ ]:
compareProtocolFeatures(d.unbound, d.bound, d.bound.feature_set.features, delta=True)

### All possible comparisons in one MEGA loop:

In [ ]:
for compare_func in (compareSingleFeatures, compareProtocolFeatures):
    for version in (data_tdict.old, data_tdict.new_reduced):
        for delta in (False, True):
            compare_func(version.unbound, version.bound, version.bound.feature_set.features, delta=delta)

## Old documentation cells

### Loading data from CSV files:

Load precomputed data about the samples encoded as tables in CSV format, and scale the data such that each feature has $\mu = 0$ and $\sigma^2 = 1$.

These data include PDB identifiers, residue numbers, feature values for each of our six features, and lastly $\Delta\Delta G$ values for each residue.

Note: $\Delta\Delta G$ values for residues in the unbound sets are actually calculated for their *bound* counterparts. 
The correspondence was inferred using a local sequence alignment between bound and unbound receptors, as detailed in the `match-bound-unbound` notebook.

### Setup learning:

Use the bound feature array as $X$, our feature data, and scale it such that each feature has $\mu = 0$ and $\sigma^2 = 1$

Load $\Delta\Delta G$ values for all residues in the bound set, and define binary classification (`y`) as "binder" when a residue has $\Delta\Delta G > 0$.

Load a list of PDB IDs for each of the samples in the data set, i.e. with length the same as `y`. This is used later in `LeaveOneLabelOut` cross-validation, to make sure that all the residues from any one protein are in the same subset of the samples (i.e. fold).

### Clustering comparison

Goal:: write 4 paragraphs about the SVM section: Q&A

check ddg cutoff

look at false positives from ConSurf

Visualize examples of Dima's table

In [ ]:
import config

In [ ]:
import peptalk

In [ ]:
conf = delta_configs[0].interactiveTree()
probs = config.predictClassifier(conf)[:,1]
preds = config.predictClassifier(conf, proba=False)
ddgs_df = conf.test_set.label_data_df
probs_df = pd.DataFrame(data=np.c_[preds, probs], index=conf.test_set.label_data_df.index, columns=['pred','prob']) 

In [ ]:
def cluster_ddg_recall(cluster, ddgs):
    return ddgs[cluster].sum() / ddgs.sum()

In [ ]:
@memory.cache
def test_clustering(conf, const_size=None):
    probs = config.predictClassifier(conf)[:,1]
    preds = config.predictClassifier(conf, proba=False)
    ddgs_df = conf.test_set.label_data_df
    probs_df = pd.DataFrame(data=np.c_[preds, probs], index=ddgs_df.index, columns=['pred','prob']) 
    
    cluster_results = {}
    for pdbid, df in probs_df.groupby(level=0):
        #print pdbid,
        svm_result = peptalk.PeptalkResult(pdbid=pdbid, preds=df.pred, confidence=df.prob)
        
        dbscan_cls = svm_result.cluster_dbscan()
        #print pdbid, dbscan_cls
        cl = dbscan_cls[0] if dbscan_cls else []
        
        prob_cl_size = const_size if const_size else len(cl)
        prob_cl_df = df.sort(column='prob', ascending=False).head(prob_cl_size).sort()
        prob_cl = prob_cl_df['prob'].index.get_level_values(1).tolist()
        #print cl, prob_cl
        ddgs = ddgs_df.ix[pdbid]
        cluster_results[pdbid] = (cluster_ddg_recall(cl, ddgs), cluster_ddg_recall(prob_cl, ddgs))
    print ""
    return cluster_results

In [ ]:
@memory.cache
def summarize(conf):
    cluster_results = test_clustering(conf, const_size=None)
    d = pd.DataFrame(cluster_results.values(), index=cluster_results.keys(), columns=['clustered','nonclustered'])
    return (d > .5).sum() / float(len(d))

In [ ]:
summ = pd.concat([summarize(c) for c in single_protocol_configs], keys=[c.title for c in single_protocol_configs])

In [ ]:
summ.unstack()

In [ ]:
d = pd.DataFrame(cluster_results.values(), index=cluster_results.keys(), columns=['len','clustered','nonclustered'])
d.clustered - d.nonclustered
d.ix[(d.clustered > d.nonclustered) & (d.clustered!=0)]
#d.ix[d.len <2]
pl.hist([d[col] for col in d.columns[1:]], 
        label=d.columns[1:].tolist(),
        alpha=.8,
        bins=linspace(0, 1, num=11, endpoint=True),
        )
pl.legend(loc='upper right')
pl.xticks(linspace(0, 1, num=11, endpoint=True))
pl.show()

## Recursive feature elimination

In [ ]:
from sklearn.feature_selection import RFECV, RFE

In [ ]:
!head -1 unboundMat

In [ ]:
rfe = RFE(estimator=classifier, n_features_to_select=3, step=1)
rfe.fit(X_train, y_train)

In [ ]:
print rfe.support_
print rfe.ranking_

In [ ]:
rfecv = RFECV(classifier, step=1, cv=StratifiedKFold(y_train, 3),
                loss_func=metrics.zero_one)
rfecv.fit(X_train, y_train)

In [ ]:
print "Optimal number of features : %d" % rfecv.n_features_
print "Support:", rfecv.support_
print "Feature ranking:", rfecv.ranking_

In [ ]:
pl.figure()
pl.xlabel("Number of features selected")
pl.ylabel("Cross validation score (nb of misclassifications)")
pl.plot(xrange(1, len(rfecv.cv_scores_) + 1), rfecv.cv_scores_)
pl.show()